### Homework 5: Question search engine

Remeber week01 where you used GloVe embeddings to find related questions? That was.. cute, but far from state of the art. It's time to really solve this task using context-aware embeddings.

__Warning:__ this task assumes you have seen `seminar.ipynb`!

In [1]:
%pip install sentencepiece
%pip install --upgrade transformers datasets accelerate deepspeed

import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
import datasets

import numpy as np
from tqdm import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 30.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 32.6 MB/s 
     |████████████████████████████████| 441 kB 60.4 MB/s 
     |████████████████████████████████| 148 kB 54.5 MB/s 
     |████████████████████████████████| 665 kB 64.7 MB/s 
     |████████████████████████████████| 7.6 MB 59.4 MB/s 
     |████████████████████████████████| 163 kB 70.4 MB/s 
     |████████████████████████████████| 115 kB 72.7 MB/s 
     |████████████████████████████████| 212 kB 56.4 MB/s 
     |████████████████████████████████| 127 kB 75.2 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
     |████████████████████████████████| 120 kB 68.4 MB/s 
     |████████████████████████████████| 115 kB 71.4 MB/s 
  Created wheel for deepspeed: filename=deepspeed-

### Load data and model

In [2]:
qqp = datasets.load_dataset('SetFit/qqp')
print('\n')
print("Sample[0]:", qqp['train'][0])
print("Sample[3]:", qqp['train'][3])

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/SetFit___json/SetFit--qqp-4b14bf13c8e13797/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]



Sample[0]: {'text1': 'How is the life of a math student? Could you describe your own experiences?', 'text2': 'Which level of prepration is enough for the exam jlpt5?', 'label': 0, 'idx': 0, 'label_text': 'not duplicate'}
Sample[3]: {'text1': 'What can one do after MBBS?', 'text2': 'What do i do after my MBBS ?', 'label': 1, 'idx': 3, 'label_text': 'duplicate'}


In [ ]:
model_name = "gchhablani/bert-base-cased-finetuned-qqp"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

Downloading:   0%|          | 0.00/320 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/890 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433M [00:00<?, ?B/s]

### Tokenize the data

In [3]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

In [ ]:
qqp_preprocessed = qqp.map(preprocess_function, batched=True)

  0%|          | 0/364 [00:00<?, ?ba/s]

  0%|          | 0/391 [00:00<?, ?ba/s]

  0%|          | 0/41 [00:00<?, ?ba/s]

In [ ]:
print(repr(qqp_preprocessed['train'][0]['input_ids'])[:100], "...")

[101, 1731, 1110, 1103, 1297, 1104, 170, 12523, 2377, 136, 7426, 1128, 5594, 1240, 1319, 5758, 136,  ...


### Task 1: evaluation (1 points)

We randomly chose a model trained on QQP - but is it any good?

One way to measure this is with validation accuracy - which is what you will implement next.

Here's the interface to help you do that:

In [ ]:
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=1, shuffle=False, collate_fn=transformers.default_data_collator
)

In [ ]:
for batch in val_loader:
     break  # here be your training code
print("Sample batch:", batch)

with torch.no_grad():
  predicted = model(
      input_ids=batch['input_ids'],
      attention_mask=batch['attention_mask'],
      token_type_ids=batch['token_type_ids']
  )

print('\nPrediction (probs):', torch.softmax(predicted.logits, dim=1).data.numpy())

Sample batch: {'labels': tensor([0]), 'idx': tensor([0]), 'input_ids': tensor([[  101,  2009,  1132,  2170,   118,  4038,  1177,  2712,   136,   102,
          2009,  1132,  1117, 10224,  4724,  1177,  2712,   136,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,   

In [ ]:
qqp['validation'][0]

{'text1': 'Why are African-Americans so beautiful?',
 'text2': 'Why are hispanics so beautiful?',
 'label': 0,
 'idx': 0,
 'label_text': 'not duplicate'}

__Your task__ is to measure the validation accuracy of your model.
Doing so naively may take several hours. Please make sure you use the following optimizations:

- run the model on GPU with no_grad
- using batch size larger than 1
- use optimize data loader with num_workers > 1
- (optional) use [mixed precision](https://pytorch.org/docs/stable/notes/amp_examples.html)


In [ ]:
# <A whole lot of YOUR CODE HERE>
val_loader = torch.utils.data.DataLoader(
    val_set, 
    batch_size=128, 
    num_workers=4,
    shuffle=False, 
    collate_fn=transformers.default_data_collator
)

model.to(device)
result = []

with torch.no_grad():
    with torch.autocast(device_type='cuda', dtype=torch.float16):
        for batch in tqdm(val_loader):
            predicted = model(
                input_ids=batch['input_ids'].to(device),
                attention_mask=batch['attention_mask'].to(device),
                token_type_ids=batch['token_type_ids'].to(device),
            )
            pred_labels = torch.softmax(predicted.logits, dim=1).argmax(dim=1).cpu().data.numpy()
            true_labels = batch['labels'].cpu().data.numpy()
            result += list(pred_labels == true_labels)

accuracy = np.mean(result)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 316/316 [01:22<00:00,  3.82it/s]


In [ ]:
assert 0.9 < accuracy < 0.91
print(accuracy)

0.9083601286173634


### Task 2: train the model (5 points)

For this task, you have two options:

__Option A:__ fine-tune your own model. You are free to choose any model __except for the original BERT.__ We recommend [DeBERTa-v3](https://huggingface.co/microsoft/deberta-v3-base). Better yet, choose the best model based on public benchmarks (e.g. [GLUE](https://gluebenchmark.com/)).

You can write the training code manually or use transformers.Trainer (see [this example](https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification)). Please make sure that your model's accuracy is at least __comparable__ with the above example for BERT.


__Option B:__ compare at least 3 pre-finetuned models (in addition to the above BERT model). For each model, report (1) its accuracy, (2) its speed, measured in samples per second in your hardware setup and (3) its size in megabytes. Please take care to compare models in equal setting, e.g. same CPU / GPU. Compile your results into a table and write a short (~half-page on top of a table) report, summarizing your findings.

## Option A

In [4]:
# <A whole lot of your code goes here>
model_name = "microsoft/deberta-v3-base"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/579 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:447: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.classifier.weight', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a

In [5]:
qqp_deberta = qqp.map(preprocess_function, batched=True)

  0%|          | 0/364 [00:00<?, ?ba/s]

  0%|          | 0/391 [00:00<?, ?ba/s]

  0%|          | 0/41 [00:00<?, ?ba/s]

In [9]:
metric = datasets.load_metric('accuracy')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = transformers.TrainingArguments(
    fp16=True,
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    learning_rate=5e-05,
    evaluation_strategy="steps",
    eval_steps=2000,
    logging_steps=2000,
    save_strategy="epoch",
    metric_for_best_model="accuracy",
)

trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=qqp_deberta["train"],
    eval_dataset=qqp_deberta["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using cuda_amp half precision backend


In [10]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: text1, idx, label_text, text2. If text1, idx, label_text, text2 are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 363846
  Num Epochs = 2
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 11372
/usr/local/lib/python3.7/dist-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:746: UserWarning: To copy con

Step,Training Loss,Validation Loss,Accuracy
2000,0.292500,0.238154,0.898788
4000,0.241700,0.223243,0.908335
6000,0.207700,0.229161,0.913777
8000,0.152200,0.213972,0.919021
10000,0.143400,0.203147,0.921618


The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: text1, idx, label_text, text2. If text1, idx, label_text, text2 are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 40430
  Batch size = 64
/usr/local/lib/python3.7/dist-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:746: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  scale, dtype=query_layer.dtype
/usr/local/lib/python3.7/dist-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:829: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sour

TrainOutput(global_step=11372, training_loss=0.19916941521665055, metrics={'train_runtime': 8317.597, 'train_samples_per_second': 87.488, 'train_steps_per_second': 1.367, 'total_flos': 4.78668109446697e+16, 'train_loss': 0.19916941521665055, 'epoch': 2.0})

#### Save the fine-tuned model to Google Drive

In [ ]:
model.save_pretrained('model')

Configuration saved in model/config.json
Model weights saved in model/pytorch_model.bin


In [ ]:
!zip -r model.zip model

  adding: model/ (stored 0%)
  adding: model/config.json (deflated 53%)
  adding: model/pytorch_model.bin (deflated 13%)


In [13]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
!cp model.zip drive/MyDrive/model.zip

### Task 3: try the full pipeline (2 points)

Finally, it is time to use your model to find duplicate questions.
Please implement a function that takes a question and finds top-5 potential duplicates in the training set. For now, it is fine if your function is slow, as long as it yields correct results.

Showcase how your function works with at least 5 examples.

In [3]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [4]:
!cp drive/MyDrive/model.zip model.zip
!unzip model.zip

Archive:  model.zip
   creating: model/
  inflating: model/config.json       
  inflating: model/pytorch_model.bin  


In [5]:
MODEL_PATH = 'model'
model_name = "microsoft/deberta-v3-base"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/579 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:447: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [30]:
from itertools import chain, compress

def get_candidates(question, train_questions, length_delta=0.1):
    min_length, max_length = (int(len(question) * (1 - length_delta)), 
                              int(len(question) * (1 + length_delta)))
    mask = map(lambda x: min_length <= len(x) <= max_length, train_questions)
    candidates = list(compress(train_questions, mask))
    return candidates

def preprocess(examples):
    return tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=128, truncation=True
    )

def create_dataset(question, candidates):
    dataset_question = datasets.Dataset.from_dict({'text1': [question]*len(candidates), 'text2': candidates})
    dataset = dataset_question.map(preprocess)
    return dataset

def get_probabilities_duplicate(dataset, model):
    loader = torch.utils.data.DataLoader(
        dataset, 
        batch_size=128, 
        num_workers=4,
        shuffle=False, 
        collate_fn=transformers.default_data_collator
    )
    model.to(device)
    probabilities = []
    with torch.no_grad():
        with torch.autocast(device_type='cuda', dtype=torch.float16):
            for batch in tqdm(loader):
                predicted = model(
                    input_ids=batch['input_ids'].to(device),
                    attention_mask=batch['attention_mask'].to(device),
                    token_type_ids=batch['token_type_ids'].to(device),
                )
                probabilities_duplicate = torch.softmax(predicted.logits, dim=1)[:,1].cpu().data.numpy()
                probabilities += list(probabilities_duplicate)
    return probabilities

def get_top_n_nearest(candidates, probabilities, n=5):
    return sorted(zip(candidates, probabilities), 
                  key=lambda x: x[1], 
                  reverse=True)[:n]

def find_duplicate_questions(question, train_questions, model):
    candidates = get_candidates(question, train_questions)
    print(f'All questions: {len(train_questions)}, selected candidates: {len(candidates)}')
    dataset = create_dataset(question, candidates)
    probabilities = get_probabilities_duplicate(dataset, model)
    return get_top_n_nearest(candidates, probabilities)

In [28]:
%%time
train_questions = list(chain.from_iterable([[t['text1'], t['text2']] for t in qqp['train']]))
print('Union of all questions from train:', len(train_questions))
train_questions = list(set(train_questions))
print('Dropped duplicated texts:', len(train_questions))

Union of all questions from train: 727692
Dropped duplicated texts: 493874
CPU times: user 44.3 s, sys: 375 ms, total: 44.6 s
Wall time: 48 s


### Searching for duplicates in optimized way (with candidates short-list)

In [31]:
question_1 = "Why is this sky so beautiful?"
duplicates_question_1 = find_duplicate_questions(question_1, train_questions, model)
duplicates_question_1

All questions: 493874, selected candidates: 32376


  0%|          | 0/32376 [00:00<?, ?ex/s]

100%|██████████| 253/253 [01:35<00:00,  2.64it/s]


[('Why is that the sky is so blue?', 0.87966454),
 ('Why is the sky blue on Earth?', 0.5863748),
 ('Why is the sky blue color?', 0.3986113),
 ('Do you think she is beautiful?', 0.0038844766),
 ('Why is the sky blue and green?', 0.0025065753)]

In [32]:
question_2 = "What does it mean?"
duplicates_question_2 = find_duplicate_questions(question_2, train_questions, model)
duplicates_question_2

All questions: 493874, selected candidates: 4331


  0%|          | 0/4331 [00:00<?, ?ex/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
  0%|          | 0/34 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:746: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  scale, dtype=query_layer.dtype
/usr/local/lib/python3.7/dist-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:829: UserWarning: To copy construct from a tensor, it is recommended to use sourceTens

[('What is the mean?', 0.96017486),
 ('What does it mean?', 0.9587699),
 ('What does ╥_╥ mean?', 0.5687452),
 ('What does 呵呵 mean?', 0.55549556),
 ('What does 枯萎 mean?', 0.39814314)]

In [33]:
question_3 = "Why is the Machine Learning so difficult to learn?"
duplicates_question_3 = find_duplicate_questions(question_3, train_questions, model)
duplicates_question_3

All questions: 493874, selected candidates: 99251


  0%|          | 0/99251 [00:00<?, ?ex/s]

100%|██████████| 776/776 [04:48<00:00,  2.69it/s]


[('How do I learn machine learning and from where?', 0.094761446),
 ('How do I start learning data structures and algorithms?', 0.043568816),
 ('What do I start with for learning machine learning?', 0.041423477),
 ('How do I start learning about data analytics?', 0.028167473),
 ('How do I start learning data analytics step by step?', 0.025347175)]

In [34]:
question_4 = "Who is Jack Nicholson"
duplicates_question_4 = find_duplicate_questions(question_4, train_questions, model)
duplicates_question_4

All questions: 493874, selected candidates: 11648


  0%|          | 0/11648 [00:00<?, ?ex/s]

100%|██████████| 91/91 [00:34<00:00,  2.62it/s]


[('Who is Jack Rogers?', 0.0003807271),
 ('Who can beat jon jones?', 0.0002482611),
 ('Who is John Abraham?', 0.00022254554),
 ('Who is Jackie Chan?', 0.00018595057),
 ('What is a jamabandi?', 0.0001816438)]

In [35]:
question_5 = "What can I present to my girlfriend for a birthday?"
duplicates_question_5 = find_duplicate_questions(question_5, train_questions, model)
duplicates_question_5

All questions: 493874, selected candidates: 106436


  0%|          | 0/106436 [00:00<?, ?ex/s]

100%|██████████| 832/832 [05:11<00:00,  2.67it/s]


[('What gift can I give to my girlfriend on her birthday?', 0.98175323),
 ('What I should gift to my girlfriend on her birthday?', 0.9652504),
 ('What should I gift my crush for her birthday?', 0.91655886),
 ('What gift should I give my crush on her birthday?', 0.8823265),
 ('Which gift should I give to a girl on her birthday?', 0.8226882)]

### Searching for duplicates in naive way (loop over all questions) - only for the last example

In [36]:
question = "What can I present to my girlfriend for a birthday?"
dataset = create_dataset(question, train_questions)
probabilities = get_probabilities_duplicate(dataset, model)
duplicates_question = get_top_n_nearest(train_questions, probabilities)
duplicates_question

  0%|          | 0/493874 [00:00<?, ?ex/s]

100%|██████████| 3859/3859 [23:41<00:00,  2.71it/s]


[('Gift for gf on birthday?', 0.9983223),
 ('What is best gifts to give my girlfriend on her birthday?', 0.988425),
 ('What gift can I give to my girlfriend on her birthday?', 0.98175323),
 ('What is the best gift to gift for a girlfriend on her birthday?',
  0.9736196),
 ('What are some great suggestions to gift my girlfriend for her birthday?',
  0.971537)]

__Bonus:__ for bonus points, try to find a way to run the function faster than just passing over all questions in a loop. For isntance, you can form a short-list of potential candidates using a cheaper method, and then run your tranformer on that short list. If you opted for this solution, please keep both the original implementation and the optimized one - and explain briefly what is the difference there.

---

### Bonus solution: boosted the speed with short-list of candidates

Instead of simple loop over all questions from train, I've implemented the pre-filtering based on the __length__ of the question. Reasonably, the most similar questions should have the similar lengths. Default parameter for length delta: __±10%__ of the original question's length.

Based on the last 5-th question I've compared the speed:
* Optimized: 00:43 + 05:11 ~ __6 mins__
* Naive: 04:27 + 23:41 ~ __28 mins__

And we can see, the resulted Top-5 questions are almost the same, besides the short ones and long ones. It could be additionally adjusted using words intersection or quicker embedding similarity, for example.